In [9]:
from typing import NamedTuple, Tuple
import seaborn as sns
import matplotlib as mpl
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from scipy.interpolate import RegularGridInterpolator
import cv2 as cv
import pandas as pd
import os
from skimage import io, img_as_float
from skimage.metrics import structural_similarity as ssim

In [10]:
mpl.rcParams['figure.dpi'] = 300
sns.set_theme()
sns.set_style("ticks")

# Fundamental functions

In [3]:
def undistort(dc: np.ndarray, k: float, yx: np.ndarray) -> np.ndarray:
    r2 = ((yx - dc) ** 2).sum(axis=1, keepdims=True)
    vu = dc + (yx - dc) / (1 + k * r2)
    return vu


def redistort(
    dc: np.ndarray,  # centre point, *2
    k: float,        # distort parameter aka. lambda
    vu: np.ndarray,  # non-distorted, n * 2
) -> np.ndarray:     # n * 2

    # Extract components for clarity
    xb = vu[:, 0]
    yb = vu[:, 1]
    x0 = dc[0]
    y0 = dc[1]

    # Calculate the radial distance rc for each point
    rc = np.sqrt((xb - x0)**2 + (yb - y0)**2)

    # Ensure that rc is not zero to avoid division by zero
    rc_safe = np.where(rc == 0, 1e-8, rc)

    # Calculate the distorted coordinates using the provided formula
    factor = (1 - np.sqrt(1 - 4 * k * rc_safe**2)) / (2 * k * rc_safe**2)
    x_distorted = x0 + (xb - x0) * factor
    y_distorted = y0 + (yb - y0) * factor

    # Combine the distorted x and y into a single array
    yx = np.vstack((x_distorted, y_distorted)).T

    return yx

        

def setup_transform(dc: np.ndarray, k: float, mn: Tuple[int, int], scale: float):
    ylong = np.arange(0, mn[0], scale) # Generate values from 0 to mn[0]-1 with a step of "scale"
    xlong = np.arange(0, mn[1], scale)
    yxrect_d3 = np.stack(np.meshgrid(xlong, ylong), axis=-1)
    yxrect = yxrect_d3.reshape((-1, 2))[:, ::-1]

    interp_grid = tuple(dim.ravel() for dim in np.indices(dimensions=mn, sparse=True))

    return {
        'mn_high': yxrect_d3.shape[:2],
        'interp_grid': interp_grid,
        'yxdistort': redistort(dc=dc, k=k, vu=yxrect),
        'yxfixed': undistort(dc=dc, k=k, yx=yxrect),
        'rect_axes': (ylong, xlong),
    }


def distort_image(transform, image: np.ndarray) -> np.ndarray:
    distort_interp = RegularGridInterpolator(
        points=transform['interp_grid'], values=image, bounds_error=False, method='linear'
    )
    distorted = distort_interp(transform['yxdistort'])
    return distorted.reshape((*transform['mn_high'], -1))


def undistort_image(transform, image: np.ndarray) -> np.ndarray:
    undistort_interp = RegularGridInterpolator(
        points=transform['rect_axes'], values=image, bounds_error=False, method='linear'
    )
    undistorted = undistort_interp(transform['yxfixed'])
    return undistorted.reshape((*transform['mn_high'], -1))

def estimate_resolution_loss(mn: Tuple[int, int], dc: np.ndarray, k: float) -> float:
    n = 256
    middle_strip = np.empty((n, 2))
    middle_strip[:, 0] = np.linspace(0, mn[0] - 1, n) # give me n values start from 0 to mn[0] - 1
    middle_strip[:, 1] = mn[1] / 2

    distorted = redistort(dc=dc, k=k, vu=middle_strip)
    y, x = distorted.T
    y = y[np.isfinite(y)]

    res_reduce = mn[0] / (y.max() - y.min())
    return res_reduce

In [4]:
def distort_given_k_and_image(k: float, clean_image: np.ndarray) -> np.ndarray:

    # Calculate image dimensions and distortion center
    mn = clean_image.shape[:2]
    
    dc = 0.5 * np.array(mn) + 0.1
    
    # Estimate the resolution reduction
    res_reduce = estimate_resolution_loss(mn=mn, dc=dc, k=k)
    
    # Setup transformation
    transform = setup_transform(mn=mn, dc=dc, k=k, scale=res_reduce)
    
    # Normalize the image to range [0, 1]
    clean_image_normalised = clean_image / 255.0 if np.max(clean_image) > 1 else clean_image
    
    # Undistort the image
    distorted_image_normalised = distort_image(transform, clean_image_normalised)
    
    # Rescale the image back to [0, 255]
    distorted_image = (distorted_image_normalised * 255).astype(np.uint8)
    
    return distorted_image


def crop_largest_square_inside_round_shape(image):
    """
    Crop the image to the largest square inside the central round-shaped content,
    ensuring the square is entirely within the non-black content.
    
    Parameters:
    image (numpy.ndarray): An (n, n, 3) array representing the image.
    
    Returns:
    numpy.ndarray: The cropped image.
    """
    # Get the dimensions of the image
    n = image.shape[0]
    
    # Find the non-black pixel coordinates
    non_black_pixels = np.argwhere(image.sum(axis=-1) != 0)
    
    # Calculate the center of the bounding circle (mean of non-black pixels)
    center = non_black_pixels.mean(axis=0).astype(int)
    
    # Calculate the radius of the bounding circle (max distance from center to non-black pixel)
    distances = np.linalg.norm(non_black_pixels - center, axis=1)
    radius = int(distances.max())
    
    # Side length of the largest square that fits inside the circle
    side_length = int(np.sqrt(2) * radius)
    
    # Ensure the side_length is even
    if side_length % 2 != 0:
        side_length -= 1
    
    # Calculate the top-left corner of the square
    half_side = side_length // 2
    start_row = center[0] - half_side
    start_col = center[1] - half_side
    
    # Ensure the crop area is within the image boundaries
    start_row = max(0, start_row)
    start_col = max(0, start_col)
    end_row = min(n, start_row + side_length)
    end_col = min(n, start_col + side_length)
    
    # Crop the image
    cropped_image = image[start_row:end_row, start_col:end_col, :]
    
    return cropped_image

def undistort_given_k_and_image(mn, k: float, distorted_image: np.ndarray) -> np.ndarray:
    
    dc = 0.5 * np.array(mn) + 0.1
    
    # Estimate the resolution reduction
    res_reduce = estimate_resolution_loss(mn=mn, dc=dc, k=k)
    
    # Setup transformation
    transform = setup_transform(mn=mn, dc=dc, k=k, scale=res_reduce)
    
    # Normalize the image to range [0, 1]
    distorted_image_normalized = distorted_image / 255.0 if np.max(distorted_image) > 1 else distorted_image
    
    # Undistort the image
    undistorted_image_normalized = undistort_image(transform, distorted_image_normalized)
    
    # Rescale the image back to [0, 255]
    undistorted_image = (undistorted_image_normalized * 255).astype(np.uint8)
    
    return undistorted_image

def calculate_ssim(image1, image2):
    
    # Convert images to grayscale (SSIM is often calculated on grayscale images)
    gray_image1 = cv.cvtColor(image1, cv.COLOR_BGR2GRAY)
    gray_image2 = cv.cvtColor(image2, cv.COLOR_BGR2GRAY)
    
    # Ensure the images are the same size
    if gray_image1.shape != gray_image2.shape:
        raise ValueError("Input images must have the same dimensions.")
    
    # Calculate SSIM
    ssim_index, _ = ssim(gray_image1, gray_image2, full=True)
    
    return ssim_index

def calculate_psnr(image1, image2):
    
    # Ensure the images are the same size
    if image1.shape != image2.shape:
        raise ValueError("Input images must have the same dimensions.")
    
    # Calculate MSE (Mean Squared Error)
    mse = np.mean((image1 - image2) ** 2)
    
    if mse == 0:
        return float('inf')  # Infinite PSNR if images are identical
    
    # Calculate PSNR
    max_pixel_value = 255.0
    psnr_value = 20 * np.log10(max_pixel_value / np.sqrt(mse))
    
    return psnr_value

# The entire platform folder

In [11]:
df = pd.read_csv("df_platform_classreg_report.csv")
df_SSIM_PSNR = df[["id1","id2","k_gt","k_preds"]]
df_SSIM_PSNR

,id1,id2,k_gt,k_preds
0,1000000,2000000,0.000001,0.000008
1,1000000,2000001,0.000002,0.000009
2,1000000,2000002,0.000004,0.000010
3,1000000,2000003,0.000005,0.000010
4,1000000,2000004,0.000006,0.000011
...,...,...,...,...
9895,1000099,2009895,0.000095,0.000091
9896,1000099,2009896,0.000097,0.000096
9897,1000099,2009897,0.000098,0.000093
9898,1000099,2009898,0.000099,0.000094


## Rectify

In [12]:
clean_folder_path = "clean_platform_images"
distorted_crop_folder_path = "distorted_platform_images_crop"
distorted_uncrop_folder_path = "distorted_platform_images_uncrop"
rectified_kgt_output_folder_path = "rectified_kgt_platform_images"
rectified_kpred_output_folder_path = "rectified_kpred_platform_images"

In [13]:
n_c = (256,256)    
m = 0.5 * np.array(n_c) + 0.1 

for id2, k_gt in df_SSIM_PSNR[['id2', 'k_gt']].values:
    distorted_image_path = os.path.join(distorted_uncrop_folder_path, f"{int(id2)}.jpg")
    scale = estimate_resolution_loss(n_c, m, k_gt)        
    distorted_image = np.array(Image.open(distorted_image_path))
    n_r = int(np.ceil(n_c[0] / scale))                   
    distorted_image_resized = cv.resize(distorted_image, (n_r, n_r))
    rectified_image = undistort_given_k_and_image(n_c, k_gt, distorted_image_resized)  # Assuming k_pred is k_gt here
    rectified_image_resize = cv.resize(rectified_image, (256,256))
    
    # Define the output path and save the image
    output_image_path = os.path.join(rectified_kgt_output_folder_path, f"{int(id2)}_rectified.jpg")
    cv.imwrite(output_image_path, cv.cvtColor(rectified_image_resize, cv.COLOR_RGB2BGR))

for id2, k_pred in df_SSIM_PSNR[['id2', 'k_preds']].values:
    distorted_image_path = os.path.join(distorted_uncrop_folder_path, f"{int(id2)}.jpg")
    scale = estimate_resolution_loss(n_c, m, k_pred)        
    distorted_image = np.array(Image.open(distorted_image_path))
    n_r = int(np.ceil(n_c[0] / scale))                   
    distorted_image_resized = cv.resize(distorted_image, (n_r, n_r))
    rectified_image = undistort_given_k_and_image(n_c, k_pred, distorted_image_resized)  # Assuming k_pred is k_gt here
    rectified_image_resize = cv.resize(rectified_image, (256,256))
    
    # Define the output path and save the image
    output_image_path = os.path.join(rectified_kpred_output_folder_path, f"{int(id2)}_rectified.jpg")
    cv.imwrite(output_image_path, cv.cvtColor(rectified_image_resize, cv.COLOR_RGB2BGR))

/var/folders/yb/q8pvpkcj757ft6nlg4_hj5080000gn/T/ipykernel_5510/4172737284.py:19: RuntimeWarning: invalid value encountered in sqrt
  ) + 0.5 * (vu - dc) * (1 - np.sqrt(1 - 4 * inner))) / inner


/var/folders/yb/q8pvpkcj757ft6nlg4_hj5080000gn/T/ipykernel_5510/4172737284.py:19: RuntimeWarning: invalid value encountered in sqrt
  ) + 0.5 * (vu - dc) * (1 - np.sqrt(1 - 4 * inner))) / inner


## SSIM, PSNR for gt

In [14]:
SSIM_list = []
PSNR_list = []

for id1, id2 in df_SSIM_PSNR[['id1', 'id2']].values:
    
    clean_img_path = os.path.join(clean_folder_path, f"{int(id1)}.jpg")
    rectified_img_path = os.path.join(rectified_kgt_output_folder_path, f"{int(id2)}_rectified.jpg")

    clean_img = cv.imread(clean_img_path)
    rectified_img = cv.imread(rectified_img_path)
    
    ssim_index = calculate_ssim(clean_img, rectified_img)
    psnr_value = calculate_psnr(clean_img, rectified_img)

    SSIM_list.append(ssim_index)
    PSNR_list.append(psnr_value)

In [15]:
df_SSIM_PSNR = df_SSIM_PSNR.copy(deep=True)
df_SSIM_PSNR["SSIM_k_gt"] = SSIM_list
df_SSIM_PSNR["PSNR_k_gt"] = PSNR_list

df_SSIM_PSNR

,id1,id2,k_gt,k_preds,SSIM_k_gt,PSNR_k_gt
0,1000000,2000000,0.000001,0.000008,0.855321,31.017823
1,1000000,2000001,0.000002,0.000009,0.799195,30.501225
2,1000000,2000002,0.000004,0.000010,0.856989,31.018381
3,1000000,2000003,0.000005,0.000010,0.825526,30.698095
4,1000000,2000004,0.000006,0.000011,0.818894,30.627956
...,...,...,...,...,...,...
9895,1000099,2009895,0.000095,0.000091,0.625710,31.798450
9896,1000099,2009896,0.000097,0.000096,0.628794,31.857443
9897,1000099,2009897,0.000098,0.000093,0.588723,31.545784
9898,1000099,2009898,0.000099,0.000094,0.622114,31.817276


## SSIM, PSNR for pred

In [16]:
SSIM_list = []
PSNR_list = []

for id1, id2 in df_SSIM_PSNR[['id1', 'id2']].values:
    
    clean_img_path = os.path.join(clean_folder_path, f"{int(id1)}.jpg")
    rectified_img_path = os.path.join(rectified_kpred_output_folder_path, f"{int(id2)}_rectified.jpg")

    clean_img = cv.imread(clean_img_path)
    rectified_img = cv.imread(rectified_img_path)
    
    ssim_index = calculate_ssim(clean_img, rectified_img)
    psnr_value = calculate_psnr(clean_img, rectified_img)

    SSIM_list.append(ssim_index)
    PSNR_list.append(psnr_value)

In [17]:
df_SSIM_PSNR = df_SSIM_PSNR.copy(deep=True)
df_SSIM_PSNR["SSIM_k_pred"] = SSIM_list
df_SSIM_PSNR["PSNR_k_pred"] = PSNR_list

df_SSIM_PSNR

,id1,id2,k_gt,k_preds,SSIM_k_gt,PSNR_k_gt,SSIM_k_pred,PSNR_k_pred
0,1000000,2000000,0.000001,0.000008,0.855321,31.017823,0.372834,28.780699
1,1000000,2000001,0.000002,0.000009,0.799195,30.501225,0.377111,28.843122
2,1000000,2000002,0.000004,0.000010,0.856989,31.018381,0.392664,28.899507
3,1000000,2000003,0.000005,0.000010,0.825526,30.698095,0.410941,28.983109
4,1000000,2000004,0.000006,0.000011,0.818894,30.627956,0.401315,28.939866
...,...,...,...,...,...,...,...,...
9895,1000099,2009895,0.000095,0.000091,0.625710,31.798450,0.532135,31.641092
9896,1000099,2009896,0.000097,0.000096,0.628794,31.857443,0.629731,31.943067
9897,1000099,2009897,0.000098,0.000093,0.588723,31.545784,0.525166,31.535412
9898,1000099,2009898,0.000099,0.000094,0.622114,31.817276,0.516202,31.573079
